In [ ]:
from matplotlib import pyplot as plt
import nlopt
import numpy as np
import kinpy as kp
import os

from omniisaacgymenvs.data_types import geometry_utils
from omniisaacgymenvs.data_types import se3

In [ ]:
#urdf_path = '~/Downloads/data/hand/aSampleForearmURDF/aSampleForearm.urdf'
left_right = 'left'
urdf_path = '~/Downloads/data/hand/psyonic/ability_hand_' + left_right + '.urdf'
chain = kp.build_chain_from_urdf(open(os.path.expanduser(urdf_path), 'rb').read())
tip_joints = {
  'thumb_anchor_frame': ['thumb_q1', 'thumb_q2'],
  'index_anchor_frame': ['index_q1', 'index_q2'],
  'middle_anchor_frame': ['middle_q1', 'middle_q2'],
  'ring_anchor_frame': ['ring_q1', 'ring_q2'],
  'pinky_anchor_frame': ['pinky_q1', 'pinky_q2']
}
joint_indices = {name: index for index, name in enumerate(chain.get_joint_parameter_names())}
finger_tips = [
  'thumb_anchor_frame',
  'index_anchor_frame',
  'middle_anchor_frame',
  'ring_anchor_frame'
]
finger_chains = [kp.chain.SerialChain(
  chain, tip_frame, 'thumb_base_frame') for tip_frame in finger_tips]
mimic_config = {
  'index_q2': ('index_q1', 1.05851325, 0.72349796),
  'middle_q2': ('middle_q1', 1.05851325, 0.72349796),
  'ring_q2': ('ring_q1', 1.05851325, 0.72349796),
  'pinky_q2': ('pinky_q1', 1.05851325, 0.72349796),
}
mimic_joints = {}
for name, source in mimic_config.items():
  mimic_joints[joint_indices[name]] = (joint_indices[source[0]], source[1], source[2])
d_vector_d_tip_positions = np.array([
  [1, 0, 0, 0],
  [0, 1, 0, 0],
  [0, 0, 1, 0],
  [0, 0, 0, 1],
  [-1, 1, 0, 0],
  [-1, 0, 1, 0],
  [-1, 0, 0, 1],
  [0, -1, 1, 0],
  [0, -1, 0, 1],
  [0, 0, -1, 1]]
)

def add_debug_frame(frame: se3.Transform,
                    parent_body: int = -1,
                    parent_link: int = -1,
                    line_length: float = 0.5,
                    line_width: float = 3,
                    duration: float = 100,
                    ):
  eye = np.identity(3)
  vec_start = frame.translation
  for i in range(3):
    trans = se3.Transform(xyz=line_length * eye[i])
    vec_end = (frame * trans).translation
    p.addUserDebugLine(vec_start, vec_end,
                        lineColorRGB=eye[i],
                        parentObjectUniqueId=parent_body,
                        parentLinkIndex=parent_link,
                        lineWidth=line_width,
                        lifeTime=duration)

In [ ]:
unit_length = 0.09473151311686484
def vector_length(vector):
  return np.linalg.norm(vector)

def normalize(vector):
  return vector / np.clip(vector_length(vector), 1e-10, 1e10)

def weight_and_scale(vector, vector_index):
  length_threshold = 0.1 / unit_length
  length = vector_length(vector)
  antipodal_set = set([4, 5, 6])
  primary_set = set([7, 8, 9])
  if length > length_threshold:
    return 1, 1.6 * length  # length #
  elif vector_index in antipodal_set:
    return 200, 1e-4  # length #
  elif vector_index in primary_set:
    return 400, 3e-2  # length #
  else:
    return 1, 1.6 * length
  
def human_points_to_vectors(points):
  vectors = [
    points[4, :] - points[0, :],
    points[8, :] - points[0, :],
    points[12, :] - points[0, :],
    points[16, :] - points[0, :],
    points[8, :] - points[4, :],
    points[12, :] - points[4, :],
    points[16, :] - points[4, :],
    points[12, :] - points[8, :],
    points[16, :] - points[8, :],
    points[16, :] - points[12, :]
  ]
  return vectors

def robot_config_to_vectors(joint_positions):
  tip_positions = []
  for index, finger_tip_frame in enumerate(finger_tips):
    finger_joints = [joint_positions[joint_indices[joint_name]]
                     for joint_name in tip_joints[finger_tip_frame]]
    tip_positions.append(finger_chains[index].forward_kinematics(finger_joints).pos / unit_length)

  vectors = [
    tip_positions[0],
    tip_positions[1],
    tip_positions[2],
    tip_positions[3],
    tip_positions[1] - tip_positions[0],
    tip_positions[2] - tip_positions[0],
    tip_positions[3] - tip_positions[0],
    tip_positions[2] - tip_positions[1],
    tip_positions[3] - tip_positions[1],
    tip_positions[3] - tip_positions[2]
  ]
  return vectors

def update_jacobian_for_mimic_joints(jacobian, mimic_joints):
  for joint_index, source in mimic_joints.items():
    source_index, multiplier, _ = source
    jacobian[:, source_index] = jacobian[:, joint_index] * multiplier + jacobian[:, source_index]
    jacobian[:, joint_index] = 0
  return jacobian

def compute_jacobians(joint_positions):
  jacobians = []
  for index, finger_tip_frame in enumerate(finger_tips):
    finger_joint_indices = [joint_indices[joint_name]
                            for joint_name in tip_joints[finger_tip_frame]]
    finger_joints = joint_positions[finger_joint_indices]
    jac = np.zeros((6, len(joint_positions)))
    jac[:, finger_joint_indices] = finger_chains[index].jacobian(finger_joints)
    update_jacobian_for_mimic_joints(jac, mimic_joints)
    jacobians.append(jac)
  return jacobians

In [ ]:
import os
import pybullet as p
import pybullet_data

physicsClient = p.connect(p.GUI)#or p.DIRECT for non-graphical version
p.setAdditionalSearchPath(os.path.expanduser('~/Downloads/data/hand/psyonic')) #used by loadURDF
p.setGravity(0,0,-10)
robot_id = p.loadURDF(
  os.path.expanduser('~/Downloads/data/hand/psyonic/ability_hand_' + left_right + '.urdf'),
  [0,0,0], p.getQuaternionFromEuler([0,0,0]), useFixedBase=True)
p.resetDebugVisualizerCamera(cameraDistance=1.0, cameraYaw=200.8, cameraPitch=-57.4, cameraTargetPosition=[0.09, -0.28, -0.54])

xyzw = p.getBasePositionAndOrientation(robot_id)[1]
world_t_base = se3.Transform(xyz=p.getBasePositionAndOrientation(robot_id)[0], rot=[xyzw[3], xyzw[0], xyzw[1], xyzw[2]])

idx = 6
xyzw = p.getLinkState(robot_id, idx)[5]
base_t_thumb_base = world_t_base.inverse() * se3.Transform(xyz=p.getLinkState(robot_id, idx)[4], rot=[xyzw[3], xyzw[0], xyzw[1], xyzw[2]])
# add_debug_frame(base_t_thumb_base, robot_id, line_length=0.05, line_width=0.02)
# for idx in [9, 12, 15, 21]:
#   xyzw = p.getLinkState(robot_id, idx)[5]
#   world_t_link = world_t_base.inverse() * se3.Transform(xyz=p.getLinkState(robot_id, idx)[4], rot=[xyzw[3], xyzw[0], xyzw[1], xyzw[2]])
#   add_debug_frame(world_t_link, robot_id, line_length=0.03, line_width=0.03)

In [ ]:
if left_right == "left":
  human_points = np.array([
  ## left
[0.28720974922180176,0.8708093762397766,0.3143763542175293],                            
[-0.12651261687278748,0.7349244952201843,0.05995972454547882],                          
[-0.36603623628616333,0.45278191566467285,-0.03126854449510574],                        
[-0.6309131979942322,0.1829824596643448,-0.1702186018228531],                           
[-0.8438302874565125,-0.13528841733932495,-0.17903482913970947],                        
[-0.2836613059043884,0.03581162542104721,0.018209077417850494],                         
[-0.3006921112537384,-0.3133622705936432,-0.1753331571817398],                          
[-0.40445154905319214,-0.2089175134897232,-0.44700461626052856],                        
[-0.4755593538284302,-0.01989281363785267,-0.6883588433265686],                                                                                                                 
[-8.366570769524628e-10,7.362022103052368e-11,6.826117449065805e-11],                   
[-0.016884777694940567,-0.304980993270874,-0.2808392643928528],                                                                                                                 
[-0.057547762989997864,-0.16373211145401,-0.5634378790855408],                          
[-0.2741575241088867,0.06048767268657684,-0.7267991900444031],                                                                                                                  
[0.2034456878900528,0.0875338464975357,-0.049468837678432465],                          
[0.19146330654621124,-0.14211268723011017,-0.32897454500198364],                                                                                                                
[0.08237076550722122,-0.0800984725356102,-0.5235435962677002],                          
[-0.08545099943876266,0.078017957508564,-0.8117814064025879],                                                                                                                   
[0.43864205479621887,0.27189236879348755,-0.1458372175693512],                          
[0.4172156751155853,-0.011291861534118652,-0.4611600935459137],                                                                                                                 
[0.30150067806243896,0.029656387865543365,-0.6703757643699646],                         
[0.1978302299976349,0.161203995347023,-0.7841989994049072],    
#     [-0.14793530106544495,0.7267894148826599,0.5503167510032654],  
#   [0.024055711925029755,0.5519598722457886,0.47814440727233887], 
#   [0.21352136135101318,0.30227985978126526,0.5160688757896423], 
#   [0.2699916362762451,-0.09295934438705444,0.3404589295387268],   
#   [0.32913506031036377,-0.18698163330554962,0.27393269538879395],
#   [0.1356365978717804,-0.06729891151189804,0.026648811995983124], 
#   [0.12202458828687668,-0.40975186228752136,-0.023529455065727234],
#   [0.17576287686824799,-0.643936038017273,-0.03044189140200615],  
#   [0.29381847381591797,-0.8803104758262634,-0.1100316047668457],  
#   [-6.841113231459417e-10,-1.4498957590092232e-10,-1.0137358508188754e-08],
#   [-0.04843224585056305,-0.42310985922813416,-0.15389062464237213],
#   [-0.04069826751947403,-0.6622974276542664,-0.17490702867507935],   
#   [0.04702276736497879,-0.9689635634422302,-0.13562990725040436], 
#   [-0.15367238223552704,0.03149377554655075,0.051093313843011856], 
#   [-0.21014076471328735,-0.2870909571647644,-0.07116362452507019], 
#   [-0.2287592887878418,-0.5698741674423218,-0.12310448288917542], 
#   [-0.15643265843391418,-0.8305444717407227,-0.038768500089645386], 
#   [-0.30132558941841125,0.12932449579238892,0.06958530098199844],
#   [-0.5066770315170288,-0.22816982865333557,0.02548663690686226],    
#   [-0.4999783933162689,-0.38263773918151855,-0.029648305848240852],
#   [-0.47550180554389954,-0.5455068945884705,-0.052836138755083084],
  ])

else:
  human_points = np.array([
  ## right
  [0.35925376415252686,0.7557541131973267,0.5102125406265259],
  [-0.0217706561088562,0.7466740012168884,0.27697762846946716],   
  [-0.2586385905742645,0.6049586534500122,-0.09853920340538025], 
  [-0.2637333273887634,0.44316551089286804,-0.5239014029502869],  
  [-0.09836322069168091,0.3405230641365051,-0.8661985993385315],              
  [-0.26269078254699707,0.09757312387228012,-0.04891512915492058],
  [-0.40207889676094055,-0.3215048313140869,-0.14132839441299438],            
  [-0.5895003080368042,-0.5254322290420532,-0.2374584823846817],           
  [-0.6998971700668335,-0.7604447603225708,-0.34814760088920593],                                                                                                                 
  [1.2982748209822148e-10,-9.16070552747783e-10,8.97446561509696e-10],     
  [-0.08608043193817139,-0.422415554523468,-0.14894185960292816],                                                                                                                 
  [-0.18812528252601624,-0.68717360496521,-0.25021734833717346],  
  [-0.2508523464202881,-1.0579893589019775,-0.22055749595165253],                                                                                                                 
  [0.2520734667778015,0.018618403002619743,-0.008291996084153652],  
  [0.20300228893756866,-0.3212728202342987,-0.13540016114711761],                                                                                                                 
  [0.21015995740890503,-0.5265160202980042,-0.3273022770881653], 
  [0.09568844735622406,-0.9591401815414429,-0.41275525093078613],                                                                                                                 
  [0.5308778285980225,0.13585840165615082,-0.032896988093853],   
  [0.5607253313064575,-0.20470702648162842,-0.1921987533569336],                                                                                                                  
  [0.5806171298027039,-0.4187415838241577,-0.28038501739501953], 
  [0.5910985469818115,-0.7268421649932861,-0.4496389925479889], 
  ])

human_points_tmp = human_points.copy()
human_points_tmp = human_points_tmp - human_points_tmp[0, :]
thumb_base_t_wrist = se3.Transform(xyz=[24.0476665e-3, 3.78124745e-3, 32.32964923e-3]).inverse()
robot_t_human_base = se3.Transform(
  xyz=(world_t_base * base_t_thumb_base * thumb_base_t_wrist).translation,
  rot=np.radians([-110, 0, 0])
)
human_points = geometry_utils.transform_points(human_points_tmp.transpose(),
                                                    robot_t_human_base).transpose()

def augment_reduced_joints(reduced_joint_positions):
  return np.array([0, 0, 0, 0, 0, 0,
                   reduced_joint_positions[0],
                   reduced_joint_positions[0] * 1.05851325 + 0.72349796,
                   reduced_joint_positions[1],
                   reduced_joint_positions[1] * 1.05851325 + 0.72349796,
                   reduced_joint_positions[2],
                   reduced_joint_positions[2] * 1.05851325 + 0.72349796,
                   reduced_joint_positions[2],
                   reduced_joint_positions[2] * 1.05851325 + 0.72349796,
                   reduced_joint_positions[3],
                   reduced_joint_positions[4]])

def weight_and_scale(vector, vector_index):
  length_threshold = 0.1 / unit_length
  length = vector_length(vector)
  antipodal_set = set([4, 5, 6])
  primary_set = set([7, 8, 9])
  if length > length_threshold:
    return 1, 1.6 * length  # length #
  elif vector_index in antipodal_set:
    return 200, 1e-4  # length #
  elif vector_index in primary_set:
    return 400, 3e-2  # length #
  else:
    return 1, 1.6 * length

def weight_and_scale(vector, vector_index):
  vector_norm = vector_length(human_vectors[vector_index])
  antipodal_set = set([4, 5, 6])
  primary_set = set([7, 8, 9])
  if vector_index in antipodal_set:
    return 20, vector_norm
  elif vector_index in primary_set:
    return 100, vector_norm
  else:
    return 1, vector_norm

def metric(reduced_joint_positions, gradient):
  human_vectors = human_points_to_vectors(human_points)
  cost = 0
  num_dof = 16
  joint_positions = augment_reduced_joints(reduced_joint_positions)
  vectors = -np.array(robot_config_to_vectors(joint_positions))
  new_grad = np.zeros((num_dof, ))
  jacobians = compute_jacobians(joint_positions)
  for vector_index, vector in enumerate(vectors):
    weight, scale = weight_and_scale(vector, vector_index)
    diff = vector - scale * normalize(human_vectors[vector_index])
    cost += weight * np.linalg.norm(diff) ** 2
    d_vector_d_joint_position = np.zeros((3, num_dof))
    for tip_index in range(d_vector_d_tip_positions.shape[1]):
      d_vector_d_joint_position += d_vector_d_tip_positions[
        vector_index, tip_index] * jacobians[tip_index][:3, :]
    new_grad += weight * np.dot(diff, d_vector_d_joint_position)
  cost = 2.5e-3 * np.linalg.norm(joint_positions) ** 2 + 0.5 * cost
  new_grad += 5e-3 * joint_positions
  if gradient.size > 0:
    gradient[:] = new_grad[[6, 8, 10, 14, 15]]
#   print(reduced_joint_positions)
  print(cost)

  return cost

human_vectors = human_points_to_vectors(human_points)

# GN_DIRECT
opt = nlopt.opt(nlopt.GN_DIRECT, 5)  # try different optimizers
opt.set_lower_bounds([0, 0, 0, -2.0943951, 0])
opt.set_upper_bounds([2.0943951, 2.0943951, 2.0943951, 0, 2.0943951])
opt.set_min_objective(metric)
opt.set_ftol_rel(1e-3)
opt.set_maxtime(1000)
theta = opt.optimize(np.array([1.0, 1.0, 1.0, -1.0, 1.0]))
minf = opt.last_optimum_value()
print(theta)
print(minf)
print(opt.last_optimize_result())

In [ ]:
use_gt = False
constructed = np.array([0., 0., 0., -0.8, 1.3])

jps = np.zeros((22))
if use_gt:
  jps[[7, 10, 13, 16, 19, 20]] = constructed[[0, 1, 2, 2, 3, 4]]
else:
  jps[[7, 10, 13, 16, 19, 20]] = theta[[0, 1, 2, 2, 3, 4]]

jps[[8, 11, 14, 17]] = jps[[7, 10, 13, 16]] * 1.05851325 + 0.72349796
for i in range(22):
  p.resetJointState(robot_id, i, jps[i])

tmp_grad = np.array([0.0] * 5)
print(metric(constructed, tmp_grad))
print(tmp_grad)
print(metric(theta, tmp_grad))
print(tmp_grad)

In [ ]:
# debugging

human_points_plot = human_points.copy()
human_vectors_plot = human_points_to_vectors(human_points_plot)

if use_gt:
  plot_joints = constructed.copy()
else:
  plot_joints = augment_reduced_joints(theta)

tip_positions = []
for index, finger_tip_frame in enumerate(finger_tips):
  finger_joints = [plot_joints[joint_indices[joint_name]]
                   for joint_name in tip_joints[finger_tip_frame]]
  tip_positions.append(finger_chains[index].forward_kinematics(finger_joints).pos)
base_t_xyz = np.array([
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  tip_positions[0],
  tip_positions[0],
  tip_positions[0],
  tip_positions[1],
  tip_positions[1],
  tip_positions[2],
])
start_xyz = geometry_utils.transform_points(
  base_t_xyz.transpose(), world_t_base * base_t_thumb_base).transpose()
diff_xyz = np.array(robot_config_to_vectors(plot_joints)) * unit_length
end_xyz = geometry_utils.transform_points(
  (base_t_xyz + diff_xyz).transpose(), world_t_base * base_t_thumb_base).transpose()

colors = [[0, 0, 0]] * 4 + [[1, 0, 0]] * 3 + [[0, 1, 0]] * 2 + [[0, 0, 1]] 
for i in range(0, start_xyz.shape[0]):
  p.addUserDebugLine(start_xyz[i, :], end_xyz[i, :], lineColorRGB=[0, 0, 1], lifeTime=30)

base_t_xyz = human_points[[0, 0, 0, 0, 4, 4, 4, 8, 8, 12], :] * unit_length
start_xyz = base_t_xyz
diff_xyz = np.array(human_vectors) * unit_length
end_xyz = base_t_xyz + diff_xyz


colors = [[0, 0, 0]] * 4 + [[1, 0, 0]] * 3 + [[0, 1, 0]] * 2 + [[0, 0, 1]] 
for i in range(0, start_xyz.shape[0]):
  p.addUserDebugLine(start_xyz[i, :], end_xyz[i, :], lineColorRGB=[1, 0, 0], lifeTime=30)

In [ ]:
print(np.array(human_points_to_vectors(human_points)))
print(np.array(robot_config_to_vectors(augment_reduced_joints(theta))))
print(np.array(robot_config_to_vectors(constructed)))
# %matplotlib qt
# fig = plt.figure()
# ax = fig.add_subplot(projection='3d')
# colors = (np.array([list(range(20, -1, -1)), [0] * 21, list(range(0, 21))]).transpose() / 20)
# colors = ['k'] + ['r'] * 4 + ['g'] * 4 + ['b'] * 4 + ['y'] * 4 + ['c'] * 4
# to_plot = np.array(human_points_to_vectors(human_points))
# to_plot2 = - np.array(robot_config_to_vectors(constructed))
# for i in range(21):#plot each point + it's index as text above
#   ax.scatter(to_plot[i,0],to_plot[i,1],to_plot[i,2],color=colors[i], marker='o')
#   ax.scatter(to_plot2[i,0],to_plot2[i,1],to_plot2[i,2],color=colors[i], marker='+')
#   ax.text(human_points[i,0],human_points[i,1],human_points[i,2],  '%s' % (str(i)), size=10, zorder=1,color='k')
# # ax.quiver(positions[:, 0], positions[:, 1], positions[:, 2], normals[:, 0], normals[:, 1], normals[:, 2], length=0.02, normalize=True)
# ax.set_xlabel('X Label')
# ax.set_ylabel('Y Label')
# ax.set_zlabel('Z Label')
# # ax.set_xlim([-0.1, 0.1])
# # ax.set_ylim([-0.1, 0.1])
# # ax.set_zlim([-0.1, 0.1])

In [ ]:
p.disconnect()

In [ ]:
# %matplotlib qt
# fig = plt.figure()
# ax = fig.add_subplot(projection='3d')
# colors = (np.array([list(range(20, -1, -1)), [0] * 21, list(range(0, 21))]).transpose() / 20)
# colors = ['k'] + ['r'] * 4 + ['g'] * 4 + ['b'] * 4 + ['y'] * 4 + ['c'] * 4
# # ax.scatter(locs[:, 0], locs[:, 1], locs[:, 2], marker=, c=colors, linewidths=10)
# for i in range(21):#plot each point + it's index as text above
#   ax.scatter(human_points[i,0],human_points[i,1],human_points[i,2],color=colors[i])
#   ax.text(human_points[i,0],human_points[i,1],human_points[i,2],  '%s' % (str(i)), size=10, zorder=1,color='k')
# # ax.quiver(positions[:, 0], positions[:, 1], positions[:, 2], normals[:, 0], normals[:, 1], normals[:, 2], length=0.02, normalize=True)
# ax.set_xlabel('X Label')
# ax.set_ylabel('Y Label')
# ax.set_zlabel('Z Label')
# # ax.set_xlim([-0.1, 0.1])
# # ax.set_ylim([-0.1, 0.1])
# # ax.set_zlim([-0.1, 0.1])